In [ ]:
import numpy as np
import collections
import torch
from torch.autograd import Variable
import torch.optim as optim

import rnn

start_token = 'G'
end_token = 'E'
batch_size = 64

device = torch.device("cuda")
device

device(type='cuda')

In [1]:
def process_poems1(file_name):
    """

    :param file_name:
    :return: poems_vector  have tow dimmention ,first is the poem, the second is the word_index
    e.g. [[1,2,3,4,5,6,7,8,9,10],[9,6,3,8,5,2,7,4,1]]

    """
    poems = []
    with open(file_name, "r", encoding='utf-8', ) as f:
        for line in f.readlines():
            try:
                title, content = line.strip().split(':')
                # content = content.replace(' ', '').replace('，','').replace('。','')
                content = content.replace(' ', '')
                if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content or \
                                start_token in content or end_token in content:
                    continue
                if len(content) < 5 or len(content) > 80:
                    continue
                content = start_token + content + end_token
                poems.append(content)
            except ValueError as e:
                print("error")
                pass
    # 按诗的字数排序
    poems = sorted(poems, key=lambda line: len(line))
    # print(poems)
    # 统计每个字出现次数
    all_words = []
    for poem in poems:
        all_words += [word for word in poem]
    counter = collections.Counter(all_words)  # 统计词和词频。
    count_pairs = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*count_pairs)
    words = words[:len(words)] + (' ',)
    word_int_map = dict(zip(words, range(len(words))))
    poems_vector = [list(map(word_int_map.get, poem)) for poem in poems]
    return poems_vector, word_int_map, words

def process_poems2(file_name):
    """
    :param file_name:
    :return: poems_vector  have tow dimmention ,first is the poem, the second is the word_index
    e.g. [[1,2,3,4,5,6,7,8,9,10],[9,6,3,8,5,2,7,4,1]]

    """
    poems = []
    with open(file_name, "r", encoding='utf-8', ) as f:
        # content = ''
        for line in f.readlines():
            try:
                line = line.strip()
                if line:
                    content = line.replace(' '' ', '').replace('，','').replace('。','')
                    if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content or \
                                    start_token in content or end_token in content:
                        continue
                    if len(content) < 5 or len(content) > 80:
                        continue
                    # print(content)
                    content = start_token + content + end_token
                    poems.append(content)
                    # content = ''
            except ValueError as e:
                # print("error")
                pass
    # 按诗的字数排序
    poems = sorted(poems, key=lambda line: len(line))
    # print(poems)
    # 统计每个字出现次数
    all_words = []
    for poem in poems:
        all_words += [word for word in poem]
    counter = collections.Counter(all_words)  # 统计词和词频。
    count_pairs = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*count_pairs)
    words = words[:len(words)] + (' ',)
    word_int_map = dict(zip(words, range(len(words))))
    poems_vector = [list(map(word_int_map.get, poem)) for poem in poems]
    return poems_vector, word_int_map, words

In [2]:
def generate_batch(batch_size, poems_vec, word_to_int):
    n_chunk = len(poems_vec) // batch_size
    x_batches = []
    y_batches = []
    for i in range(n_chunk):
        start_index = i * batch_size
        end_index = start_index + batch_size
        x_data = poems_vec[start_index:end_index]
        y_data = []
        for row in x_data:
            y  = row[1:]
            y.append(row[-1])
            y_data.append(y)
        """
        x_data             y_data
        [6,2,4,6,9]       [2,4,6,9,9]
        [1,4,2,8,5]       [4,2,8,5,5]
        """
        # print(x_data[0])
        # print(y_data[0])
        # exit(0)
        x_batches.append(x_data)
        y_batches.append(y_data)
    return x_batches, y_batches

In [3]:
def run_training():
    # 处理数据集
    # poems_vector, word_to_int, vocabularies = process_poems2('./tangshi.txt')
    poems_vector, word_to_int, vocabularies = process_poems1('./poems.txt')
    # 生成batch
    print("finish  loadding data")
    BATCH_SIZE = 100

    torch.manual_seed(5)
    word_embedding = rnn.word_embedding( vocab_length= len(word_to_int) + 1 , embedding_dim= 100).to(device)
    rnn_model = rnn.RNN_model(
        batch_sz = BATCH_SIZE,
        vocab_len = len(word_to_int) + 1,
        word_embedding = word_embedding,
        embedding_dim= 100, 
        lstm_hidden_dim=128
    ).to(device)

    # optimizer = optim.Adam(rnn_model.parameters(), lr= 0.001)
    optimizer=optim.RMSprop(rnn_model.parameters(), lr=0.01)

    loss_fun = torch.nn.NLLLoss()
    # rnn_model.load_state_dict(torch.load('./poem_generator_rnn'))  # if you have already trained your model you can load it by this line.

    for epoch in range(30):
        batches_inputs, batches_outputs = generate_batch(BATCH_SIZE, poems_vector, word_to_int)
        n_chunk = len(batches_inputs)
        for batch in range(n_chunk):
            batch_x = batches_inputs[batch]
            batch_y = batches_outputs[batch] # (batch , time_step)

            loss = 0
            for index in range(BATCH_SIZE):
                x = np.array(batch_x[index], dtype = np.int64)
                y = np.array(batch_y[index], dtype = np.int64)
                x = Variable(torch.from_numpy(np.expand_dims(x,axis=1))).to(device)
                y = Variable(torch.from_numpy(y )).to(device)
                pre = rnn_model(x)
                loss += loss_fun(pre , y)
                if index == 0:
                    _, pre = torch.max(pre, dim=1)
                    print('prediction', pre.data.tolist()) # the following three lines can print the output and the prediction
                    print('b_y       ', y.data.tolist())   # And you need to take a screenshot and then past it to your homework paper.
                    print('*' * 30)
            loss  = loss  / BATCH_SIZE
            if batch % 10 == 0:
                print("epoch  ",epoch,'batch number',batch,"loss is: ", loss.data.tolist())
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm(rnn_model.parameters(), 1)
            optimizer.step()

            if batch % 20 ==0:
                torch.save(rnn_model.state_dict(), './poem_generator_rnn')
                print("finish  save model")

NameError: name 'start_token' is not defined

In [ ]:
def to_word(predict, vocabs):  # 预测的结果转化成汉字
    sample = np.argmax(predict)

    if sample >= len(vocabs):
        sample = len(vocabs) - 1

    return vocabs[sample]


def pretty_print_poem(poem):  # 令打印的结果更工整
    shige=[]
    for w in poem:
        if w == start_token or w == end_token:
            break
        shige.append(w)
    poem_sentences = poem.split('。')
    for s in poem_sentences:
        if s != '' and len(s) > 10:
            print(s + '。')


def gen_poem(begin_word):
    # poems_vector, word_int_map, vocabularies = process_poems2('./tangshi.txt')  #  use the other dataset to train the network
    poems_vector, word_int_map, vocabularies = process_poems1('./poems.txt')
    word_embedding = rnn.word_embedding(vocab_length=len(word_int_map) + 1, embedding_dim=100).to(device)
    rnn_model = rnn.RNN_model(batch_sz=64, vocab_len=len(word_int_map) + 1, word_embedding=word_embedding,
                                   embedding_dim=100, lstm_hidden_dim=128).to(device)

    rnn_model.load_state_dict(torch.load('./poem_generator_rnn', map_location=device, weights_only=True))

    # 指定开始的字

    poem = start_token+begin_word
    word = begin_word
    while word != end_token:
        input = np.array([word_int_map[w] for w in poem],dtype= np.int64)
        input = Variable(torch.from_numpy(input)).to(device)
        output = rnn_model(input, is_test=True)
        word = to_word(output.data.tolist()[-1], vocabularies)
        poem += word
        # print(word)
        # print(poem)
        if len(poem) > 100:
            break
    return poem[1:]

In [ ]:
pretty_print_poem(gen_poem("日"))
pretty_print_poem(gen_poem("红"))
pretty_print_poem(gen_poem("山"))
pretty_print_poem(gen_poem("夜"))
pretty_print_poem(gen_poem("湖"))
pretty_print_poem(gen_poem("海"))
pretty_print_poem(gen_poem("月"))
pretty_print_poem(gen_poem("君"))

inital  linear weight 
日日朝云处，东南望不成。
不闻天下事，不见一生人。
日月浮云上，风光月上春。
风前不见处，风雨夜相如。
日月浮云色，风光夜未成。
何如一夜月，应见一年春。
inital  linear weight 
红树生时不复，金天不见出门前。
不如天下客，不见此时心。
日月长无事，风光一夜风。
风霜有一夜，风雨夜相如。
不是东堂客，何人更有时。
inital  linear weight 
山中有何日，一夜千山春。
风月千门色，山中万岁生。
云开山色色，风急夜长清。
风月春光晚，风清月上春。
风光浮不得，山水夜风霜。
为有东堂客，何人更不行。
inital  linear weight 
夜来风雨后，日日上清光。
万事无人见，无心不有无。
一千千万岁，何处是闲人。
树色浮生水，山光夜雨收。
春风有一色，风雨夜相如。
inital  linear weight 
湖上风光出，山光日日开。
不如天下事，不见此时归。
日月长无事，风清万事闲。
风光浮不出，风雨夜相飞。
日月浮光上，风光带雨中。
此时无一事，应见一年春。
inital  linear weight 
海上山中月，春风一日长。
一年无处处，一日一时春。
风月浮云上，山花月上春。
风前无处客，风雨夜相迎。
风月几时客，山川一夜风。
何当更何事，应见一生人。
inital  linear weight 
月色风霜夜，清风出水清。
不如天下事，不见此时难。
日月长无事，风光一夜风。
风霜有清日，风雨夜相如。
风月千门下，山川一夜风。
何如一夜月，不见一生人。
inital  linear weight 
君子有秦处，今朝不见春。
一千千万岁，何处是闲人。
树色浮分水，山光夜夜长。
风霜有此日，山色几时春。
树色清风下，山云月上春。
何如一夜月，应是此时生。


In [ ]:
run_training()  # 如果不是训练阶段 ，请注销这一行 。 网络训练时间很长。

In [ ]:
def check_poem_format(file_name):
    """
    检查文件中每一行是否符合“标题:诗句”的格式，
    并输出有问题的行号和原因。
    """
    forbidden_chars = ["_", "(", "（", "《", "[", "G", "E"]  # 可根据需要调整

    with open(file_name, "r", encoding="utf-8") as f:
        line_number = 0
        error_found = False
        for line in f:
            line_number += 1
            line = line.strip()
            if not line:  # 跳过空行
                continue

            # 检查是否有且仅有一个冒号
            parts = line.split(":")
            if len(parts) != 2:
                print(f"Line {line_number}: 格式错误，冒号数量不为1 -> {line}")
                error_found = True
                continue

            title, content = parts
            if not title:
                print(f"Line {line_number}: 标题为空 -> {line}")
                error_found = True
            if not content:
                print(f"Line {line_number}: 内容为空 -> {line}")
                error_found = True

        if not error_found:
            print("所有行格式均符合要求！")

# 示例调用：
check_poem_format("poems.txt")

所有行格式均符合要求！
